In [1]:
# ignore warning
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# working with dates
from datetime import datetime

# to evaluated performance using rmse
from sklearn.metrics import mean_squared_error
from math import sqrt 

# for tsa 
import statsmodels.api as sm

# holt's linear trend model. 
from statsmodels.tsa.api import Holt

In [2]:
df = pd.read_csv('cpi.csv')

In [3]:
df.head()

,year,period,label,all_items_value,all_items_ monthly_change,apparel_value,apparel_monthly_change,energy_value,energy_monthly_change,food_value,food_monthly_change,gas_value,gas_monthly_change,medical_value,medical_monthly_change,transportation_value,transportation_monthly_change
0,1957,M01,1957 Jan,27.67,0.1,44.3,0.0,21.3,0.0,28.4,-0.4,NaN,NaN,16.7,0.6,23.5,0.9
1,1957,M02,1957 Feb,27.80,0.5,44.3,0.0,21.4,0.5,28.7,1.1,NaN,NaN,16.7,0.0,23.7,0.9
2,1957,M03,1957 Mar,27.86,0.2,44.5,0.5,21.5,0.5,28.6,-0.3,NaN,NaN,16.8,0.6,23.7,0.0
3,1957,M04,1957 Apr,27.93,0.3,44.4,-0.2,21.6,0.5,28.6,0.0,NaN,NaN,16.9,0.6,23.8,0.4
4,1957,M05,1957 May,28.00,0.3,44.5,0.2,21.6,0.0,28.7,0.3,NaN,NaN,16.9,0.0,23.9,0.4


In [4]:
# Check null values
df.isnull().sum()

year                               0
period                             0
label                              0
all_items_value                    0
all_items_ monthly_change          0
apparel_value                      0
apparel_monthly_change             0
energy_value                       0
energy_monthly_change              0
food_value                         0
food_monthly_change                0
gas_value                        120
gas_monthly_change               120
medical_value                      0
medical_monthly_change             0
transportation_value               0
transportation_monthly_change      0
dtype: int64

In [5]:
# Convert label date to datetime
df['label'] = pd.to_datetime(df['label'], infer_datetime_format=True)

In [6]:
# Convert year to datetime
df['year'] =  pd.to_datetime(df['year']).dt.to_period('Y')

In [7]:
# Rename period to month
df = df.rename(columns={'period': 'month'})

In [8]:
# Convert period to datetime
df['month'] = df['month'].str.replace('M', '')

In [9]:
df.dtypes

year                              period[A-DEC]
month                                    object
label                            datetime64[ns]
all_items_value                         float64
all_items_ monthly_change               float64
apparel_value                           float64
apparel_monthly_change                  float64
energy_value                            float64
energy_monthly_change                   float64
food_value                              float64
food_monthly_change                     float64
gas_value                               float64
gas_monthly_change                      float64
medical_value                           float64
medical_monthly_change                  float64
transportation_value                    float64
transportation_monthly_change           float64
dtype: object

In [10]:
df.head()

,year,month,label,all_items_value,all_items_ monthly_change,apparel_value,apparel_monthly_change,energy_value,energy_monthly_change,food_value,food_monthly_change,gas_value,gas_monthly_change,medical_value,medical_monthly_change,transportation_value,transportation_monthly_change
0,1970,01,1957-01-01,27.67,0.1,44.3,0.0,21.3,0.0,28.4,-0.4,NaN,NaN,16.7,0.6,23.5,0.9
1,1970,02,1957-02-01,27.80,0.5,44.3,0.0,21.4,0.5,28.7,1.1,NaN,NaN,16.7,0.0,23.7,0.9
2,1970,03,1957-03-01,27.86,0.2,44.5,0.5,21.5,0.5,28.6,-0.3,NaN,NaN,16.8,0.6,23.7,0.0
3,1970,04,1957-04-01,27.93,0.3,44.4,-0.2,21.6,0.5,28.6,0.0,NaN,NaN,16.9,0.6,23.8,0.4
4,1970,05,1957-05-01,28.00,0.3,44.5,0.2,21.6,0.0,28.7,0.3,NaN,NaN,16.9,0.0,23.9,0.4


## Prepare Function

In [3]:
def get_data():
    '''
    This function returns the cleaned dataset
    '''
    df = pd.read_csv('cpi.csv')
    # Convert label date to datetime
    df['label'] = pd.to_datetime(df['label'], infer_datetime_format=True)
    # Convert year to datetime
    df['year'] =  pd.to_datetime(df['year']).dt.to_period('Y')
    # Rename period to month
    df = df.rename(columns={'period': 'month'})
    # Convert period to datetime
    df['month'] = df['month'].str.replace('M', '')
    
    return df

In [4]:
df = get_data()

In [5]:
df.head()

,year,month,label,all_items_value,all_items_ monthly_change,apparel_value,apparel_monthly_change,energy_value,energy_monthly_change,food_value,food_monthly_change,gas_value,gas_monthly_change,medical_value,medical_monthly_change,transportation_value,transportation_monthly_change
0,1970,01,1957-01-01,27.67,0.1,44.3,0.0,21.3,0.0,28.4,-0.4,NaN,NaN,16.7,0.6,23.5,0.9
1,1970,02,1957-02-01,27.80,0.5,44.3,0.0,21.4,0.5,28.7,1.1,NaN,NaN,16.7,0.0,23.7,0.9
2,1970,03,1957-03-01,27.86,0.2,44.5,0.5,21.5,0.5,28.6,-0.3,NaN,NaN,16.8,0.6,23.7,0.0
3,1970,04,1957-04-01,27.93,0.3,44.4,-0.2,21.6,0.5,28.6,0.0,NaN,NaN,16.9,0.6,23.8,0.4
4,1970,05,1957-05-01,28.00,0.3,44.5,0.2,21.6,0.0,28.7,0.3,NaN,NaN,16.9,0.0,23.9,0.4


In [6]:
df.shape

(787, 17)

## Tranform Shape of DataFrame

In [12]:
df.set_index('label', inplace = True)

In [18]:
df_melt = pd.melt(df, value_vars = ['all_items_value', 'apparel_value', 'energy_value', 'food_value', 'gas_value',
                                    'medical_value', 'transportation_value'],
                 value_name = 'cpi', ignore_index = False)

In [19]:
df_melt

,variable,cpi
label,,
1957-01-01,all_items_value,27.670
1957-02-01,all_items_value,27.800
1957-03-01,all_items_value,27.860
1957-04-01,all_items_value,27.930
1957-05-01,all_items_value,28.000
...,...,...
2022-03-01,transportation_value,341.104
2022-04-01,transportation_value,351.573
2022-05-01,transportation_value,356.018


In [20]:
df_melt['date'] = df_melt.index

In [22]:
df_wide = pd.pivot(df_melt, index = 'variable', columns = 'date', values = 'cpi')

In [23]:
df_wide

date,1957-01-01,1957-02-01,1957-03-01,1957-04-01,1957-05-01,1957-06-01,1957-07-01,1957-08-01,1957-09-01,1957-10-01,...,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01
variable,,,,,,,,,,,,,,,,,,,,,
all_items_value,27.67,27.8,27.86,27.93,28.0,28.11,28.19,28.28,28.32,28.32,...,276.590,278.524,280.126,281.933,284.182,287.708,288.663,291.474,295.328,295.271
apparel_value,44.30,44.3,44.50,44.40,44.5,44.50,44.50,44.60,44.50,44.60,...,121.850,122.760,124.117,125.433,126.342,127.044,126.087,126.927,127.929,127.818
energy_value,21.30,21.4,21.50,21.60,21.6,21.60,21.50,21.40,21.40,21.40,...,255.010,261.257,263.553,265.929,275.170,305.469,297.233,308.839,332.087,316.955
food_value,28.40,28.7,28.60,28.60,28.7,28.90,29.10,29.40,29.20,29.20,...,283.912,286.050,287.440,289.959,292.912,295.809,298.379,301.879,304.867,308.220
gas_value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,287.303,300.351,304.315,301.791,321.849,380.763,357.377,372.063,413.606,381.710
medical_value,16.70,16.7,16.80,16.90,16.9,17.00,17.10,17.10,17.10,17.20,...,576.248,578.237,580.198,583.779,584.581,588.223,591.331,593.887,598.227,600.683
transportation_value,23.50,23.7,23.70,23.80,23.9,24.00,24.20,24.40,24.40,24.50,...,323.995,326.256,326.397,329.726,334.305,341.104,351.573,356.018,363.425,361.783


In [24]:
df_wide.to_csv('cpi_wide.csv')